In [1]:
%reset -f
import sys
sys.path.insert(0,'..')
# import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import utils
from wavhandler import *
from utils_train import train_test_val_split

In [2]:
data1 = Dataset('Leafminers')
data1.read(data=data1.target_classes[0], setting='psd_dB', labels='text')
data2 = Dataset('LG')
data2.read(data=data2.target_classes[0], setting='psd_dB', labels='text')
data3 = Dataset('LG')
data3.read(data=data3.target_classes[1], setting='psd_dB', labels='text')
data4 = Dataset('Pcfruit')
data4.read(data=data4.target_classes[1], setting='psd_dB', labels='text')

Data: Chicorii.
Read 1451 filenames in 0.02 seconds.
Loaded data into matrix in 0.53 seconds.
Data: LG_drosophila_10_09.
Read 5536 filenames in 0.14 seconds.
Loaded data into matrix in 1.31 seconds.
Data: LG_zapr_26_09.
Read 7210 filenames in 0.11 seconds.
Loaded data into matrix in 1.55 seconds.
Data: D. suzukii.
Read 6423 filenames in 0.05 seconds.
Loaded data into matrix in 1.35 seconds.


In [3]:
data1.clean(plot=False)
data2.clean(plot=False)
data3.clean(plot=False)
data4.clean(plot=False)

data = pd.DataFrame()
data['fnames'] = pd.concat([data1.filenames, data2.filenames, data3.filenames, data4.filenames], axis=0).reset_index(drop=True)
data['y'] = data.fnames.apply(lambda x: x.split('/')[6])
# big.dropna(how='any', axis=1, inplace=True)
data.sample(10)

,fnames,y
3151,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
15758,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
18217,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
4008,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
2445,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
8932,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
18640,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
13875,/home/kalfasyan/data/insects/Pcfruit/D. suzuki...,D. suzukii
3004,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
9231,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09


In [4]:
X,y = data.fnames, data.y

In [5]:
# X,y = data.filenames.values.reshape(-1,1), data.y.values

model_name='first_' 
setting='raw'

In [6]:
top_weights_path = TEMP_DATADIR + str(model_name) + '.h5'
targets = len(np.unique(y))

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

if setting == 'psd_dB':
    X = transform_data(X, setting=setting)

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X.tolist(),y.tolist(), random_state=0)



## MODEL

In [7]:
# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(5000, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(targets, activation='softmax'))

## Compile and fit

In [8]:
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_accuracy', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_accuracy', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]


In [10]:
import math
from utils_train import train_generator, valid_generator
target_names = list(np.unique(data.y))
batch_size = 32
model.fit_generator(train_generator(X_train, y_train, batch_size=batch_size,
                                   target_names=target_names,
                                   setting=setting),
                   steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                   epochs=100,
                   validation_data = valid_generator(X_val, y_val,
                                                    batch_size=batch_size,
                                                    target_names=target_names,
                                                    setting=setting),
                    validation_steps=int(math.ceil(float(len(X_test))/float(batch_size))),
                    callbacks = callbacks_list)

Epoch 1/100
435/436 [============================>.] - ETA: 0s - loss: 0.0759 - accuracy: 0.9733

W1002 16:15:21.626078 140203200026432 callbacks.py:990] Can save best model only with val_acc available, skipping.
W1002 16:15:21.627251 140203200026432 callbacks.py:1250] Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy
W1002 16:15:21.629468 140203200026432 callbacks.py:1824] Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr


436/436 [==============================] - 118s 270ms/step - loss: 0.0757 - accuracy: 0.9734 - val_loss: 0.9976 - val_accuracy: 0.6906
Epoch 2/100
435/436 [============================>.] - ETA: 0s - loss: 0.0597 - accuracy: 0.9777

W1002 16:17:19.717518 140203200026432 callbacks.py:990] Can save best model only with val_acc available, skipping.
W1002 16:17:19.718674 140203200026432 callbacks.py:1250] Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy
W1002 16:17:19.720137 140203200026432 callbacks.py:1824] Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr


436/436 [==============================] - 118s 271ms/step - loss: 0.0596 - accuracy: 0.9777 - val_loss: 5.4728 - val_accuracy: 0.5995
Epoch 3/100
435/436 [============================>.] - ETA: 0s - loss: 0.0520 - accuracy: 0.9825

...

W1002 16:50:00.205523 140203200026432 callbacks.py:990] Can save best model only with val_acc available, skipping.
W1002 16:50:00.206901 140203200026432 callbacks.py:1250] Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy
W1002 16:50:00.208820 140203200026432 callbacks.py:1824] Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr


436/436 [==============================] - 114s 260ms/step - loss: 0.0096 - accuracy: 0.9963 - val_loss: 8.1699 - val_accuracy: 0.4687
Epoch 20/100
405/436 [==========================>...] - ETA: 7s - loss: 0.0240 - accuracy: 0.9917

E1002 16:51:43.023529 140203200026432 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kalfasyan/miniconda3/envs/wbtf2/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-c93a4729dc6c>", line 15, in <module>
    callbacks = callbacks_list)
  File "/home/kalfasyan/miniconda3/envs/wbtf2/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py", line 1297, in fit_generator
    steps_name='steps_per_epoch')
  File "/home/kalfasyan/miniconda3/envs/wbtf2/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_generator.py", line 265, in model_iteration
    batch_outs = batch_function(*batch_data)
  File "/home/kalfasyan/miniconda3/envs/wbtf2/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py", line 973, in train_on_batch
    class_weight=class_weight, reset_metrics=reset_metrics)
  File "/home/kalfasyan/miniconda3/envs/wbtf2/lib/python3.6/site-packages/tens

KeyboardInterrupt: 

## Test model

In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate_generator(valid_generator(X_test, 
                                                    y_test, 
                                                    batch_size=batch_size, 
                                                    setting=setting, 
                                                    target_names=target_names),
        steps = int(math.ceil(float(len(X_test)) / float(batch_size))))

print('loss', loss)
print('Test accuracy:', acc)

from tensorflow.keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + "_raw_final.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)